In [36]:
import os
import mido
import numpy as np

folder_path = 'midi_files'
midi_files = [f for f in os.listdir(folder_path)]
midi_dict = {}

for file in midi_files:
    sequence = np.zeros((8,8))
    midi = mido.MidiFile(f'{folder_path}/{file}')
    step = 0
    for track in midi.tracks:
        for msg in track:
            if msg.type == 'note_on':
                note = msg.note
                sequence[step%8][(note-4)%8] = 1
                step += 1
            elif msg.type == 'note_off':
                note = msg.note
                sequence[step%8][(note-4)%8] = 0
                step += 1
    midi_dict[os.path.splitext(file)[0]] = sequence
midi_dict.keys()

dict_keys(['y4', 'x1', 'y5', 'x3', 'y7', 'y6', 'x2', 'x6', 'y2', 'y3', 'x7', 'y1', 'x5', 'x4', 'y13', 'y12', 'y10', 'y11', 'y15', 'y14', 'y16', 'x10', 'x9', 'x8', 'x11', 'x13', 'x12', 'x16', 'x15', 'y8', 'y9', 'x14'])